In [ ]:
import dapla as dp
import pandas as pd

In [ ]:
#dp.show('/felles/mock_sysselsatte')

In [ ]:
comp = dp.read_pandas('/felles/mock_sysselsatte/companies_2019_30000')
pop = dp.read_pandas('/felles/mock_sysselsatte/population_2019_30000')

In [ ]:
pop

In [ ]:
# Aldersfordeling i populasjonen
pop['age'].value_counts().sort_index().plot.line()

In [ ]:
comp

### Sletting

In [ ]:
# Droppe kolonner
comp.drop('employee_points', axis = 1)

In [ ]:
#comp

In [ ]:
# Finne duplikater på nace
comp[comp.duplicated('nace', keep = False)].sort_values('nace')

In [ ]:
# Droppe duplikater ved å bruke .drop_duplicates
comp.drop_duplicates('nace').sort_values('nace')

In [ ]:
# Droppe duplikater ved å gruppere, og beholde den første
comp.groupby('nace', as_index = False).first()
# OBS, du mister indexen på raden!!!

### Ofte kan det være smart å filtrere ned først
Men lag en kopi av orginaldatasettet før du gjør permanente endringer

In [ ]:
comp_tmp = comp

In [ ]:
comp_tmp

In [ ]:
# Enkelt filter Oslo
comp_tmp_oslo = comp_tmp[comp_tmp['region_code'] == '03']
comp_tmp_oslo

In [ ]:
# Invert filter
comp_tmp_ikkeoslo = comp_tmp[comp_tmp['region_code'] != '03']
comp_tmp_ikkeoslo = comp_tmp[~(comp_tmp['region_code'] == '03')]
comp_tmp_ikkeoslo

In [ ]:
# Enkelt filter Oslo eller Østfold
comp_tmp_osloostfold = comp_tmp[(comp_tmp['region_code'] == '03') | (comp_tmp['region_code'] == '01')]
comp_tmp_osloostfold

In [ ]:
# Isin Oslo eller Østfold
comp_tmp_osloostfold = comp_tmp[comp_tmp['region_code'].isin(['03', '01'])]
comp_tmp_osloostfold

In [ ]:
# unique
unike_regionkoder = pd.unique(comp['region_code']).tolist()
unike_regionkoder

In [ ]:
unike_regionkoder[:-1].sort()

In [ ]:
unike_regionkoder[:5]

In [ ]:
comp[comp['region_code'].isin(unike_regionkoder[:5])]

### "Kobling"

In [ ]:
# Merge (alle varibaler)
pop.merge(comp, how = 'left')
#pop.merge(comp, how = 'left', left_on = 'work_id', right_on='work_id')

In [ ]:
# Concat, slå sammen lignende ting
pop_2020 = dp.read_pandas('/felles/mock_sysselsatte/population_2020_30000')
fnr_20192020 = pd.concat([pop_2020['id'], pop['id']])
fnr_20192020

In [ ]:
len(pd.unique(fnr_20192020))

In [ ]:
comp_tmp = comp

In [ ]:
comp_tmp[comp_tmp['nace'] == '01.000u']

In [ ]:
# .loc for å sette verdier i en annen kolonne
comp_tmp.loc[comp_tmp['nace'] == '01.000u', 'employee_points'] = '0'
# Se på de som evt. ble omkodet
comp_tmp[comp_tmp['nace'] == '01.000u']

In [ ]:
# loop gjennom pandas-katalog med iterrows()
for i, row in comp.iterrows():
    wrk_id = row['work_id']
    print(wrk_id, row['nace'])
    #print(len(pop[pop['work_id'] == wrk_id]))

In [ ]:
demo_sample = comp.sample(10)
demo_sample

In [ ]:
# .iloc vs. .loc
demo_sample.iloc[0]

In [ ]:
#demo_sample.loc[0]
comp.loc[10]

### Påfør radene originaldatasettet

In [ ]:
# Bevissthet rundt radnummer / index

### Ryddighet

In [ ]:
# del for å være ryddig 
comp_tmp

In [ ]:
del comp_tmp
comp_tmp

In [ ]:
# valuation og state på dp.write_pandas()

dp.write_pandas(comp, '/felles/mock_sysselsatte/companies_2019_30000test', valuation = 'OPEN', state = 'TEMP')

# skriv dataframen tilbake til daplalagringen med en del options og path.
# Verdivurderingsscorenivå:    .option("valuation", "XXXX") = SENSITVE, SHIELDED, INTERNAL eller OPEN
# Dataferdigstillingssnivå:    .option("state", "XXXX") = RAW, INPUT, PROCESSED, OUTPUT, PRODUCT, TEMP eller OTHER
# Se kodekilde: https://github.com/statisticsnorway/dapla-catalog-protobuf/blob/master/src/main/proto/dataset.proto